In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# https://drive.google.com/file/d/1c2YtIaJkMPGZkSStlzgU-2UzZf3J0K13/view?usp=sharing

In [ ]:
# ! gdown --id 1c2YtIaJkMPGZkSStlzgU-2UzZf3J0K13

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1c2YtIaJkMPGZkSStlzgU-2UzZf3J0K13
To: /content/DS_10283_3443_mini.zip
100% 309M/309M [00:03<00:00, 83.3MB/s]


In [ ]:
# !pip install transformers datasets

In [ ]:
# !pip install phonemizer

In [ ]:
# !sudo apt-get update
# !sudo apt-get install espeak

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
espeak is already the newest version (1.48.15+dfsg-3).
0 upgraded, 0 newly installed, 0 to remove and 129 not upgraded.


In [ ]:
# pip install g2p-en

In [ ]:
# !pip install wget  # Install wget if not already installed

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=72f8d8a52b239a9549dc62fcafaa00252fc71dfa2f9eec7d5e5229f219d5e553
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import wget

In [ ]:
# Provide the URL of the dataset file you want to download
url = 'https://datashare.ed.ac.uk/download/DS_10283_3443.zip'  # Replace with your dataset URL

# Download the dataset file to the current directory in Colab
wget.download(url)
print("Download success!")

Download success!


In [ ]:
import zipfile
import os
from collections import defaultdict
from IPython.display import Audio, display
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import pandas as pd
import csv
import torch
from g2p_en import G2p
import re
import string
import tqdm

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# Replace 'filename.zip' with the name of your uploaded zip file
zip_ref = zipfile.ZipFile('/content/VCTK-Corpus-0.92.zip', 'r')
zip_ref.extractall('/content')  # Specify the folder to extract to
zip_ref.close()


In [ ]:
# Base directory
base_dir = '/content/VCTK-Corpus-0.92'  # Replace with your base directory path

# Directories for text and wav files
text_dir = os.path.join(base_dir, 'txt')
wav_dir = os.path.join(base_dir, 'wav48_silence_trimmed')

# Initialize a defaultdict to store mappings
file_mapping = defaultdict(list)

# Function to extract file names without extensions
def get_filename(file_path):
    return os.path.splitext(os.path.basename(file_path))[0]

# Traverse through text directory
for root, dirs, files in os.walk(text_dir):
    for file in files:
        if file.endswith('.txt'):  # Assuming text files end with .txt
            text_file_path = os.path.join(root, file)
            text_prefix = get_filename(text_file_path)

            # Find mic1 files corresponding to each text file
            wav_speaker_dir = os.path.join(wav_dir, os.path.basename(os.path.dirname(text_file_path)))
            if os.path.exists(wav_speaker_dir):
                wav_files = os.listdir(wav_speaker_dir)
                for wav_file in wav_files:
                    if wav_file.startswith(text_prefix) and wav_file.endswith('_mic1.flac'):
                        mic1_file_path = os.path.join(wav_speaker_dir, wav_file)
                        file_mapping[text_file_path].append(mic1_file_path)


In [ ]:
data = {'audio_file': [], 'ground_truth':[]}

# Play FLAC audio and display text
for key, value in file_mapping.items():
    for text_path, mic1_path in zip([key]*len(value), value):
        data['audio_file'].append(mic1_path)
        with open(text_path, 'r') as file:
            text_content = file.read()
            data['ground_truth'].append(text_content)

df = pd.DataFrame(data)
df.to_csv('mini_vctk_transcipts.csv', index=True)

## Preparation




In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


In [ ]:


processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")

def resample_audio(file_path, target_sample_rate=16000):
    # Load the audio file
    waveform, original_sample_rate = torchaudio.load(file_path)

    # Check if the audio needs to be resampled
    if original_sample_rate != target_sample_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=original_sample_rate, new_freq=target_sample_rate)
        waveform = resampler(waveform)

    return waveform, target_sample_rate
def speech_to_phonemes(file_path):
    # Load audio
    speech_array, sampling_rate = resample_audio(file_path)
    speech_array = speech_array.squeeze()

    # Process audio
    inputs = processor(speech_array, sampling_rate=sampling_rate, return_tensors="pt", padding=True)

    # Forward pass through the model
    with torch.no_grad():
        logits = model(inputs.input_values).logits

    # Decode the predicted IDs to phonemes
    predicted_ids = torch.argmax(logits, dim=-1)
    phonemes = processor.batch_decode(predicted_ids)

    return phonemes
# IPA to ARPABET mapping for vowels
ipa_to_arpabet_vowels = {
    'ɑ': 'AA',     # father, start
    'æ': 'AE',     # bat
    'ʌ': 'AH',     # butt
    'ɔ': 'AO',     # caught, talk
    'aʊ': 'AW',    # bout
    'ə': 'AX',     # about, syrup [1]
    'ər': 'AXR',   # letter, perceive [3]
    'eɪ': 'EY',    # bait
    'ɛ': 'EH',     # bet
    'ɝ': 'ER',     # bird, nurse
    'ɪ': 'IH',     # bit
    'i': 'IY',     # beat
    'aɪ': 'AY',    # bite
    'ɑr': 'AA R',  # barn, star [2]
    'ɔɪ': 'OY',    # boy
    'oʊ': 'OW',    # boat
    'u': 'UW',     # boot
    'ʊ': 'UH',     # book
}

# IPA to ARPABET mapping for consonants
ipa_to_arpabet_consonants = {
    'b': 'B',      # buy
    'tʃ': 'CH',    # China
    'd': 'D',      # die
    'ð': 'DH',     # thy
    'f': 'F',      # fight
    'ɡ': 'G',      # guy
    'h': 'HH',     # high
    'dʒ': 'JH',    # jive
    'k': 'K',      # kite
    'l': 'L',      # lie
    'm': 'M',      # my
    'n': 'N',      # nigh
    'ŋ': 'NG',     # sing, finger
    'ɲ': 'NX',     # canyon [3]
    'p': 'P',      # pie
    'ɹ': 'R',      # rye
    's': 'S',      # sigh
    'ʃ': 'SH',     # shy
    't': 'T',      # tie
    'θ': 'TH',     # thigh
    'v': 'V',      # vie
    'w': 'W',      # wise
    'ʍ': 'WH',     # which [without wine–whine merger]
    'j': 'Y',      # yacht
    'z': 'Z',      # zoo
    'ʒ': 'ZH',     # pleasure
}

# Merge both dictionaries into one
ipa_to_arpabet = {**ipa_to_arpabet_vowels, **ipa_to_arpabet_consonants}

# Example usage of the mapping
def convert_ipa_to_arpabet(ipa_transcription):
    # Split the transcription into individual phonemes
    # Note: for affricates and diphthongs, use a more sophisticated parsing technique
    # that recognizes multi-character IPA symbols.
    arpabet_transcription = [ipa_to_arpabet.get(phoneme, '') for phoneme in ipa_transcription.split()]
    return ' '.join(arpabet_transcription)

def clean_string(input_string):
    # Remove integers
    result = re.sub(r'\d+', '', input_string)
    # Remove punctuation
    result = result.translate(str.maketrans('', '', string.punctuation))
    # Remove trailing spaces
    result = result.strip()
    return result

In [ ]:
df = pd.read_csv('/content/mini_vctk_transcipts.csv')
df['audio_phonetics'] = None
df['audio_phonemes'] = None
df['gt_phonemes'] = None


g2p = G2p()

for i in tqdm.tqdm(range(len(df))):
  df.loc[i, 'audio_phonetics'] = list(speech_to_phonemes(df.loc[i, 'audio_file']))
  df.loc[i, 'audio_phonemes'] = convert_ipa_to_arpabet(df.loc[i, 'audio_phonetics'][0])
  df.loc[i, 'gt_phonemes'] = ' '.join(g2p(df.loc[i, 'ground_truth']))
  df.loc[i, 'gt_phonemes'] = clean_string(df.loc[i, 'gt_phonemes'])

100%|██████████| 44283/44283 [11:03:46<00:00,  1.11it/s]


In [ ]:
df.to_csv('/content/mini_vctk_p_to_p.csv', index=False)

In [ ]:
accent_df = pd.read_csv("/content/idl_speakers_accent_main.csv")

In [ ]:
split_paths = df['audio_file'].str.split('/')
df['ID'] = split_paths.apply(lambda x: x[-2])

accent_df['ID'] = accent_df['ID'].astype(str)

df = pd.merge(df, accent_df, on='ID')
df = df[['ID', 'ACCENTS', 'audio_file', 'ground_truth', 'audio_phonetics', 'audio_phonemes', 'gt_phonemes']]

In [ ]:
df.to_csv('/content/vctk_with_accent.csv', index=False)